# Hamming Diversity

__Hamming Diversity__ is calculated by keeping a count of the occurence of a token in previous time steps / groups. It penalise using same token in current group which was used in previous groups at the same time step.
See [Diverse Beam Search: Decoding Diverse Solutions from Neural Sequence Models](https://arxiv.org/pdf/1610.02424.pdf) for more details.


Args:<br>
* __diversity_penalty (`float`):__<br>
        This value is subtracted from a beam's score if it generates a token same as any beam from other group at a
        particular time. Note that `diversity_penalty` is only effective if `group beam search` is enabled. diversity_penalty parameter during beam search with Hamming Diversity can influence the diversity and creativity of the generated text.<br>
*  __num_beams (`int`):__<br>
        Number of beams used for group beam search. Increasing num_beams generally increases diversity by exploring more alternative sequences or beams in parallel. `num_beams` should be divisible by `num_beam_groups` for group beam search.<br>
* __num_beam_groups (`int`):__<br>
        Number of groups to divide `num_beams` into in order to ensure diversity among different groups of beams. Increasing num_beam_groups encourages diversity within each group, promoting varied sequences within the same group and reducing repetition across groups.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings("ignore")

In [2]:
# With limiting the generated sequence to six tokens

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
inputs = tokenizer(["Artificial intelligence is transforming industries"], return_tensors="pt")

## 1. With a moderate diversity penalty, you aim to strike a balance between generating varied outputs and maintaining coherence. The outputs might have a mix of diverse and relevant content


In [3]:
beam_ids = model.generate(inputs["input_ids"], num_beams=2, num_beam_groups = 2, diversity_penalty = 0.5, max_new_tokens=6) 
print(tokenizer.batch_decode(beam_ids, skip_special_tokens=True)[0])
#Artificial intelligence is transforming industries, and it's not just

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligence is transforming industries, and it's not just


## 2. Increasing the number of beams allows the model to explore more alternatives in parallel. However, since the beam groups remain limited, you might still observe repetition in the generated sequences.


In [4]:
beam_ids = model.generate(inputs["input_ids"], num_beams=4, num_beam_groups = 2, diversity_penalty = 0.5, max_new_tokens=6)
print(tokenizer.batch_decode(beam_ids, skip_special_tokens=True)[0])
#Artificial intelligence is transforming industries, and it's not just

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligence is transforming industries, and it's not just


## 3. With more beams, there's a higher likelihood of exploring diverse possibilities.

In [5]:
beam_ids = model.generate(inputs["input_ids"], num_beams=8, num_beam_groups = 2, diversity_penalty = 0.5, max_new_tokens=6)
print(tokenizer.batch_decode(beam_ids, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligence is transforming industries and the lives of millions of


## 4. With more beam groups, the model can explore different sequences within and across groups, increasing diversity. However, the relatively low diversity penalty might still lead to some repetition.


In [6]:
beam_ids = model.generate(inputs["input_ids"], num_beams=4, num_beam_groups = 4, diversity_penalty = 0.5, max_new_tokens=6)
print(tokenizer.batch_decode(beam_ids, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligence is transforming industries, and it's not just


## 5. A high diversity penalty promotes diversity at the expense of coherence. The generated outputs might become more fragmented and less coherent due to the aggressive avoidance of repetition

In [7]:
beam_ids = model.generate(inputs["input_ids"], num_beams=4, num_beam_groups = 4, diversity_penalty = 10.9, max_new_tokens=6)
print(tokenizer.batch_decode(beam_ids, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Artificial intelligence is transforming industries like healthcare, education, and


__Overall, these examples showcase how different combinations of num_beams, num_beam_groups, and diversity_penalty can influence the level of diversity and coherence in the generated outputs. Note the output is highly depend on the pretrained model that has been chosen.__